In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.2 MB/s 
     |████████████████████████████████| 9.3 MB 1.6 MB/s 
     |████████████████████████████████| 1.6 MB 37.3 MB/s 
     |████████████████████████████████| 108 kB 40.4 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,256,4,1,0,1,...,1,1,3,0,6.603616,5.443362,1181520,6.627552,1,0
1,144,3,4,65535,184,184,4,1,0,1,...,1,1,3,0,5.205926,2.123522,7680,5.318221,0,0
2,144,3,4,65535,184,272,5,1,0,1,...,1,1,4,0,6.238000,3.380859,57872,6.507758,1,0
3,144,3,4,65535,184,184,1,1,0,1,...,1,0,1,0,0.000000,0.000000,95616,4.575092,1,0
4,144,3,4,65535,184,224,5,1,0,1,...,1,1,4,0,6.355626,0.702621,48128,5.545531,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,216,5,1,1,1,...,1,0,5,0,6.174602,3.155928,365568,7.546568,0,1
5206,80,2,4,65535,184,256,7,1,0,1,...,1,4,3,0,0.000000,0.000000,98816,6.947195,0,1
5207,144,3,4,65535,184,216,5,1,0,1,...,1,2,3,0,6.503422,3.790871,227328,7.823114,0,1
5208,144,3,4,65535,184,248,5,1,1,1,...,1,1,4,0,6.115208,7.919091,271616,7.886012,0,1


In [6]:
from sklearn.utils import shuffle
df = shuffle(df)

In [7]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
1162,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.513550,5.889917,4691480,7.996554,1,0
2786,144,3,4,65535,184,232,6,1,1,1,...,1,1,5,0,7.863513,0.053827,322560,7.763087,0,1
4410,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,0,6.572192,2.927885,124928,7.566389,0,1
3312,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.235098,0.000000,12416,3.274689,0,1
2608,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,5.336899,0.000000,458843,7.856939,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2753,144,3,4,65535,184,256,3,1,1,1,...,1,3,0,0,0.000000,0.000000,113152,5.721998,0,1
3549,144,3,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,184832,7.938307,1,1
3741,144,3,4,65535,184,224,10,1,0,1,...,1,7,3,0,7.199737,1.284515,391168,6.919409,0,1
3520,144,3,4,65535,184,224,5,1,0,1,...,1,3,2,0,6.300803,0.000000,60928,6.827450,0,1


In [8]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [9]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [10]:
df

,index,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,1162,144,3,4,65535,184,232,4,1,0,...,1,1,3,0,6.513550,5.889917,4691480,7.996554,1,0
1,2786,144,3,4,65535,184,232,6,1,1,...,1,1,5,0,7.863513,0.053827,322560,7.763087,0,1
2,4410,144,3,4,65535,184,216,4,1,1,...,1,0,4,0,6.572192,2.927885,124928,7.566389,0,1
3,3312,144,3,4,65535,184,192,2,1,1,...,1,0,2,0,3.235098,0.000000,12416,3.274689,0,1
4,2608,144,3,4,65535,184,184,3,1,1,...,1,0,3,0,5.336899,0.000000,458843,7.856939,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,2753,144,3,4,65535,184,256,3,1,1,...,1,3,0,0,0.000000,0.000000,113152,5.721998,0,1
5206,3549,144,3,4,65535,184,256,3,1,1,...,1,2,1,1,0.000000,0.000000,184832,7.938307,1,1
5207,3741,144,3,4,65535,184,224,10,1,0,...,1,7,3,0,7.199737,1.284515,391168,6.919409,0,1
5208,3520,144,3,4,65535,184,224,5,1,0,...,1,3,2,0,6.300803,0.000000,60928,6.827450,0,1


In [11]:
df.drop(['index'],axis=1,inplace=True)

In [12]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.513550,5.889917,4691480,7.996554,1,0
1,144,3,4,65535,184,232,6,1,1,1,...,1,1,5,0,7.863513,0.053827,322560,7.763087,0,1
2,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,0,6.572192,2.927885,124928,7.566389,0,1
3,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.235098,0.000000,12416,3.274689,0,1
4,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,5.336899,0.000000,458843,7.856939,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,256,3,1,1,1,...,1,3,0,0,0.000000,0.000000,113152,5.721998,0,1
5206,144,3,4,65535,184,256,3,1,1,1,...,1,2,1,1,0.000000,0.000000,184832,7.938307,1,1
5207,144,3,4,65535,184,224,10,1,0,1,...,1,7,3,0,7.199737,1.284515,391168,6.919409,0,1
5208,144,3,4,65535,184,224,5,1,0,1,...,1,3,2,0,6.300803,0.000000,60928,6.827450,0,1


In [13]:
df_5percent=df[0:521] 


In [14]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,232,4,1,0,1,...,1,1,3,0,6.513550,5.889917,4691480,7.996554,1,0
1,144,3,4,65535,184,232,6,1,1,1,...,1,1,5,0,7.863513,0.053827,322560,7.763087,0,1
2,144,3,4,65535,184,216,4,1,1,1,...,1,0,4,0,6.572192,2.927885,124928,7.566389,0,1
3,144,3,4,65535,184,192,2,1,1,1,...,1,0,2,0,3.235098,0.000000,12416,3.274689,0,1
4,144,3,4,65535,184,184,3,1,1,1,...,1,0,3,0,5.336899,0.000000,458843,7.856939,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,6.026036,0.000000,110592,5.586142,1,0
517,144,3,4,65535,184,232,6,1,0,1,...,1,2,4,0,6.491550,0.000000,93184,7.628381,1,1
518,144,3,4,65535,184,208,5,1,1,1,...,1,0,5,1,6.309378,7.943651,403456,7.674149,0,1
519,144,3,4,65535,184,264,5,1,0,1,...,1,1,4,0,6.166167,3.882637,78632,6.232840,1,0


In [15]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (251, 69)
class 1: (270, 69)


In [16]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [17]:
df_5percent.info

<bound method DataFrame.info of      e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0     144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
1     144.0   3.0        4.0     65535.0  184.0     232.0               6.0   
2     144.0   3.0        4.0     65535.0  184.0     216.0               4.0   
3     144.0   3.0        4.0     65535.0  184.0     192.0               2.0   
4     144.0   3.0        4.0     65535.0  184.0     184.0               3.0   
..      ...   ...        ...         ...    ...       ...               ...   
516   144.0   3.0        4.0     65535.0  184.0     128.0               3.0   
517   144.0   3.0        4.0     65535.0  184.0     232.0               6.0   
518   144.0   3.0        4.0     65535.0  184.0     208.0               5.0   
519   144.0   3.0        4.0     65535.0  184.0     264.0               5.0   
520   144.0   3.0        4.0     65535.0  184.0     240.0               4.0   

     CreationYear  

In [18]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.000000,521.0,...,521.0,521.000000,521.000000,521.000000,521.000000,521.000000,5.210000e+02,521.000000,521.000000,521.000000
mean,172.827255,41.391555,28.619962,64943.554702,216.199616,224.222649,4.531670,0.996161,0.391555,1.0,...,1.0,1.262956,3.268714,0.172745,4.892416,2.399852,7.315715e+05,6.352983,0.531670,0.518234
std,771.314119,878.278881,562.839109,6062.712515,769.616782,47.251011,2.051138,0.061898,0.488567,0.0,...,0.0,1.797956,1.142181,0.378390,2.535384,2.594198,2.784234e+06,1.163335,0.499476,0.500148
min,0.000000,0.000000,0.000000,0.000000,0.000000,16.000000,1.000000,0.000000,0.000000,1.0,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.048000e+03,2.022283,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,3.000000,1.000000,0.000000,1.0,...,1.0,0.000000,3.000000,0.000000,3.793057,0.000000,6.144000e+04,5.678278,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,4.000000,1.000000,0.000000,1.0,...,1.0,1.000000,4.000000,0.000000,6.170410,1.591198,1.201040e+05,6.411989,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.0,2.000000,4.000000,0.000000,6.509409,4.560231,3.338240e+05,7.452338,1.000000,1.000000
max,17739.000000,20050.000000,12851.000000,65535.000000,17744.000000,512.000000,34.000000,1.000000,1.000000,1.0,...,1.0,31.000000,7.000000,1.000000,7.967267,7.997217,3.917324e+07,7.999230,1.000000,1.000000


In [19]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [20]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [21]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [22]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
0.0      50.0
1.0      50.0
dtype: float64

In [23]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      52.5
0.0      47.5
dtype: float64

In [24]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [25]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [26]:
circuit(weights_init, data[0])

tensor(0.06146575, requires_grad=True)

In [27]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias


In [28]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss


In [29]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [30]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [31]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.07979091, requires_grad=False), tensor(-0.35265193, requires_grad=False)], Y = -1
X = [tensor(-0.08057058, requires_grad=False), tensor(0.54041688, requires_grad=False)], Y = -1
X = [tensor(-0.08065167, requires_grad=False), tensor(-0.26046447, requires_grad=False)], Y =  1
X = [tensor(-0.08038535, requires_grad=False), tensor(-0.26235182, requires_grad=False)], Y =  1
X = [tensor(-0.08043728, requires_grad=False), tensor(-0.26219445, requires_grad=False)], Y = -1


In [32]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [33]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.2676921 | Accuracy: 0.5000000 
Iter:     2 | Cost: 1.1040364 | Accuracy: 0.4812500 
Iter:     3 | Cost: 0.9974609 | Accuracy: 0.5000000 
New best
Iter:     4 | Cost: 0.9776409 | Accuracy: 0.5812500 
Iter:     5 | Cost: 0.9652332 | Accuracy: 0.5687500 
Iter:     6 | Cost: 0.9705312 | Accuracy: 0.5125000 
New best
Iter:     7 | Cost: 0.9568094 | Accuracy: 0.7375000 
Iter:     8 | Cost: 0.9604173 | Accuracy: 0.5125000 
Iter:     9 | Cost: 0.9479826 | Accuracy: 0.5937500 
Iter:    10 | Cost: 0.9451980 | Accuracy: 0.5687500 
Iter:    11 | Cost: 0.9590894 | Accuracy: 0.5687500 
Iter:    12 | Cost: 0.9978103 | Accuracy: 0.5625000 
Iter:    13 | Cost: 1.0689719 | Accuracy: 0.5687500 
Iter:    14 | Cost: 1.1456815 | Accuracy: 0.5687500 
Iter:    15 | Cost: 1.1795455 | Accuracy: 0.5687500 
Iter:    16 | Cost: 1.1421873 | Accuracy: 0.5750000 
Iter:    17 | Cost: 1.0024546 | Accuracy: 0.5687500 
Iter:    18 | Cost: 0.9433151 | Accuracy: 0.5937500 
Iter:    19 | Cost:

In [34]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [35]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.492327376769855, Accuracy: 35.0%


In [36]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,-1.0,-1.0
2,-1.0,1.0
3,-1.0,1.0
4,1.0,-1.0
5,1.0,-1.0
6,-1.0,1.0
7,1.0,1.0
8,1.0,-1.0
9,-1.0,1.0


In [37]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.53      0.37      0.43        27
         1.0       0.19      0.31      0.24        13

    accuracy                           0.35        40
   macro avg       0.36      0.34      0.34        40
weighted avg       0.42      0.35      0.37        40

0.19047619047619047
0.3076923076923077
0.23529411764705882
0.33903133903133903
